In [23]:
import pandas as pd
import defillama_utils as dfl
import requests as r
import plotly.express as px

chain = 'Optimism'
tokens = ['USDC','USDT','DAI','SUSD','LUSD','FRAX']
tokens_str = ', '.join(tokens)
print(tokens_str)

USDC, USDT, DAI, SUSD, LUSD, FRAX


In [24]:
# protos = 'https://api.llama.fi/protocols'
# s = r.Session()
# #get all protocols
# resp = pd.DataFrame( s.get(protos).json() )[['category','slug','chainTvls']]
# display(resp)
# # extract the chain names
# # resp['chainTvls'] = resp['chainTvls'].apply(lambda x: list(x.keys()) )
# # # set a true/false if the array contains the chain we want
# # resp['contains_chain'] = resp['chainTvls'].apply(lambda x: chain_name in x)
# # # filter where we have a match
# # proto_list = resp[['slug']][resp['contains_chain'] == True]
# # # clean up
# # proto_list = proto_list.reset_index(drop=True)
# # # boom


In [25]:
slugs = dfl.get_protocols_by_chain(chain, exclude_cex = True, exclude_chain = True)
display(slugs)
pds_raw = dfl.get_range(slugs, chain)


,slug
0,curve
1,uniswap-v3
2,multichain
3,aave-v3
4,arrakis-finance
...,...
87,optinyan
88,pegasus-finance
89,juggler-red
90,bitant


In [26]:
pds = pds_raw.copy()
pds = pds[pds['token'].isin(tokens)]
pds['total_token_tvl'] = pds.groupby(['protocol', 'chain','date'])['usd_value'].\
                                transform('sum').astype(int)

pds['day_rank'] = pds.groupby(['protocol', 'chain'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
pds['usd_value_millions'] = (pds['usd_value'] / 1e6)
display(pds)

,date,token,token_value,usd_value,total_app_tvl,protocol,chain,category,total_token_tvl,day_rank,usd_value_millions
1,2022-07-17 00:00:00,FRAX,5.727694e+02,5.730323e+02,1.108283e+07,curve,Optimism,Dexes,11029294,158,5.730323e-04
2,2022-07-17 00:00:00,SUSD,2.415611e+06,2.434926e+06,1.108283e+07,curve,Optimism,Dexes,11029294,158,2.434926e+00
3,2022-07-17 00:00:00,LUSD,4.000000e-05,4.000000e-05,1.108283e+07,curve,Optimism,Dexes,11029294,158,4.000000e-11
4,2022-07-17 00:00:00,USDT,4.191069e+06,4.191538e+06,1.108283e+07,curve,Optimism,Dexes,11029294,158,4.191538e+00
5,2022-07-17 00:00:00,USDC,2.258077e+06,2.258332e+06,1.108283e+07,curve,Optimism,Dexes,11029294,158,2.258332e+00
...,...,...,...,...,...,...,...,...,...,...,...
798,2022-12-20 00:00:00,DAI,6.731880e+00,6.731880e+00,4.140639e+03,symphony-finance,Optimism,Dexes,3214,2,6.731880e-06
802,2022-12-20 01:59:35,SUSD,2.622760e+02,2.622760e+02,4.151767e+03,symphony-finance,Optimism,Dexes,3214,1,2.622760e-04
803,2022-12-20 01:59:35,USDT,1.528641e+03,1.530170e+03,4.151767e+03,symphony-finance,Optimism,Dexes,3214,1,1.530170e-03
805,2022-12-20 01:59:35,USDC,1.415335e+03,1.415335e+03,4.151767e+03,symphony-finance,Optimism,Dexes,3214,1,1.415335e-03


In [27]:
pds_latest = pds[pds['day_rank'] == 1]
pds_latest = pds_latest.sort_values(by = 'total_token_tvl', ascending=False)
display(pds_latest)

,date,token,token_value,usd_value,total_app_tvl,protocol,chain,category,total_token_tvl,day_rank,usd_value_millions
2541,2022-12-20 01:59:35,USDC,5.503336e+07,5.503336e+07,1.017413e+08,aave-v3,Optimism,Lending,65776301,1,55.033362
2543,2022-12-20 01:59:35,DAI,7.798238e+06,7.806037e+06,1.017413e+08,aave-v3,Optimism,Lending,65776301,1,7.806037
2542,2022-12-20 01:59:35,SUSD,3.595876e+05,3.595876e+05,1.017413e+08,aave-v3,Optimism,Lending,65776301,1,0.359588
2540,2022-12-20 01:59:35,USDT,2.574741e+06,2.577316e+06,1.017413e+08,aave-v3,Optimism,Lending,65776301,1,2.577316
175,2022-12-20 01:59:47,USDT,3.616400e+07,3.620016e+07,3.620016e+07,beefy,Optimism,Yield Aggregator,36200162,1,36.200162
...,...,...,...,...,...,...,...,...,...,...,...
264,2022-12-20 01:59:47,USDT,0.000000e+00,0.000000e+00,1.700384e+04,relaychain,Optimism,Bridge,0,1,0.000000
265,2022-12-20 01:59:47,DAI,0.000000e+00,0.000000e+00,1.700384e+04,relaychain,Optimism,Bridge,0,1,0.000000
1365,2022-12-20 01:59:35,USDC,0.000000e+00,0.000000e+00,3.511610e+04,elk,Optimism,Dexes,0,1,0.000000
1366,2022-12-20 01:59:35,DAI,0.000000e+00,0.000000e+00,3.511610e+04,elk,Optimism,Dexes,0,1,0.000000


In [28]:
stable_sum = pds_latest[['protocol','usd_value','usd_value_millions']].groupby(['protocol']).sum()
stable_sum = stable_sum.reset_index()
display(stable_sum)

,protocol,usd_value,usd_value_millions
0,2pi-network,7.645693e+05,0.764569
1,aave-v3,6.577630e+07,65.776302
2,acryptos,5.137706e+05,0.513771
3,aelin,2.142432e+05,0.214243
4,alchemix,1.571723e+05,0.157172
...,...,...,...
71,woofi-earn,5.346726e+05,0.534673
72,xtoken,2.886559e+05,0.288656
73,xy-finance,1.386549e+05,0.138655
74,yearn-finance,1.217759e+05,0.121776


In [29]:
# stable_sum['usd_value_millions'] = stable_sum['usd_value_millions'].map("${:,.2f}M".format)
chart_pds_latest = pds_latest[pds_latest['total_token_tvl']>1e6]
fig = px.bar(chart_pds_latest, x="protocol", y="usd_value", color="token", title="Protocol TVL by Stablecoin - > $1M in Selected Tokens\
        <br><sup>Note: if the bar is 100% USDT, that means DefiLlama doesn't have splits by token</sup>")
fig.update_layout(barmode='stack', xaxis={'categoryorder': 'total descending'})
fig.show()

fig_pie = px.pie(stable_sum, values='usd_value_millions', names='protocol', title='Share of Stablecoins by Protocol | '+tokens_str)
fig_pie.update_traces(textposition='inside', 
                        text = stable_sum['usd_value_millions'].map("${:,.2f}M".format),
                        textinfo='percent+label+text')
fig_pie.show()